In [ ]:
# 1) Clone repo (and pull Git LFS weights)
# This repo MUST contain:
#   - parser_service/
#   - qwen_adapter_files/

REPO_URL = "https://github.com/RowanFayez/routeai-nlu-service.git"
REPO_DIR = "routeai-nlu-service"

# Colab does not always have Git LFS preinstalled.
!apt-get -qq update
!apt-get -qq install git-lfs
!git lfs install

!git clone {REPO_URL} {REPO_DIR}
%cd {REPO_DIR}

# Pull LFS objects (adapter weights)
!git lfs pull
!ls

In [ ]:
# 2) Install deps
!pip -q install -r parser_service/requirements.txt
!pip -q install pyngrok
import os, requests

In [ ]:
# 3) Configure env for the service
# Point at your adapter folder in this repo
os.environ['ADAPTER_PATH'] = os.path.abspath('qwen_adapter_files')
# Base model matches adapter_config.json
os.environ['BASE_MODEL_ID'] = os.environ.get('BASE_MODEL_ID', 'unsloth/qwen2.5-3b-instruct-unsloth-bnb-4bit')
os.environ['LOAD_IN_4BIT'] = os.environ.get('LOAD_IN_4BIT', 'true')
os.environ['MAX_NEW_TOKENS'] = os.environ.get('MAX_NEW_TOKENS', '256')
print('ADAPTER_PATH=', os.environ['ADAPTER_PATH'])
print('BASE_MODEL_ID=', os.environ['BASE_MODEL_ID'])

In [ ]:
# 4) Start uvicorn (background)
# Using shell background is more reliable in Colab than subprocess piping.

!nohup python -m uvicorn parser_service.app:app --host 0.0.0.0 --port 8000 --log-level info > uvicorn.log 2>&1 &

import time, requests
from requests.exceptions import RequestException

# Give uvicorn time to import + bind the port
time.sleep(3)

last_err = None
for attempt in range(1, 21):
    try:
        r = requests.get('http://127.0.0.1:8000/health', timeout=5)
        print('health:', r.status_code, r.json())
        last_err = None
        break
    except RequestException as e:
        last_err = e
        print(f'Waiting for server... attempt {attempt}/20 ({type(e).__name__})')
        time.sleep(3)

print('Uvicorn log tail:')
!tail -n 80 uvicorn.log

# If still not up, show process info to help debug
if last_err is not None:
    print('Server did not start. Showing uvicorn process list...')
    !ps aux | grep uvicorn | head -n 20
    raise last_err

In [ ]:
# 5) Start ngrok and print PARSER_URL
# If you don't have an account, create one at https://ngrok.com and copy your auth token.

import os
from getpass import getpass
from pyngrok import ngrok

token = os.environ.get("NGROK_AUTH_TOKEN")
if not token:
    token = getpass("Paste your NGROK_AUTH_TOKEN (input hidden): ")

ngrok.set_auth_token(token)
tunnel = ngrok.connect(8000, "http")

# IMPORTANT: requests needs the raw URL string, not NgrokTunnel's repr().
PARSER_URL = getattr(tunnel, "public_url", None) or getattr(tunnel, "url", None) or str(tunnel)
print("PARSER_URL=", PARSER_URL)
print("Now paste this into your agent env as PARSER_URL")

In [ ]:
# 6) Smoke test via PUBLIC URL
import requests

health = requests.get(f"{PARSER_URL}/health", timeout=120).json()
print("health:", health)

payload = {"query": "عايز اروح من سيدي جابر لمحطة مصر"}
parsed = requests.post(f"{PARSER_URL}/parse", json=payload, timeout=240).json()
print("parse:", parsed)

## Use in  agent
Set env var in your agent runtime:
- `PARSER_URL=<the printed PARSER_URL>`
Then in `app/services/llm.py` call `POST /parse` with `{ "query": user_input }`.